In [ ]:
%pip install fbmc-linearisation-analysis

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt 
pd.set_option('display.max_rows', 500)
plt.rcParams["figure.figsize"] = [12,6]

import os
os.environ['ENTSOE_API_KEY']  =  "" # insert your ENTSO-E API key here

# FBMC - Linearisation Error in Flow Based 

In this example we will visualize the linearisation error in the flow based methodology by:
- Fetching the required raw data (observed net postions (ENTSO-E) and flow based parameters (JAO)).
- Calculating the linearised flow using flow based methedology.
- Estimate the linearisation error by comparing observed flow with the linearised flow.

#### Start with selecting a time period to analyze

In [ ]:
from datetime import date
start = date(2023, 4, 1)
end = date(2023, 5, 1)

# Retrieving Net Positions - Base case and observed

#### Fetch flow based parameters  
These contains PTDFS and Fref values for each cnec for every hour, needed for calculating the linearised flow.

In [ ]:
from fbmc_quality.jao_data.fetch_jao_data import fetch_jao_dataframe_timeseries

df_jao = fetch_jao_dataframe_timeseries(start, end)
if df_jao is None:
    raise RuntimeError("No data")

df_jao.iloc[0]

#### Calculate basecase (D-2) net positions from flow-based parameters
The net position is calculated from the flow based data by summing the cross boarder flows (fref). This is done by a bult-in function in the fbmc_quality package.

In [ ]:
from fbmc_quality.jao_data.analyse_jao_data import compute_basecase_net_pos

basecase_nps = compute_basecase_net_pos(start, end)

if basecase_nps is None:
    raise RuntimeError("No data")

basecase_nps.head()

#### Fetch observed net positions from ENTSO-E

In [ ]:
from fbmc_quality.entsoe_data import fetch_net_position_from_crossborder_flows

observed_nps = fetch_net_position_from_crossborder_flows(start, end)

if observed_nps is None:
    raise RuntimeError("No data")

observed_nps.head()

#### Plot base case (D-2) and observed net position for a bidding zone

In [ ]:
bidding_zone = 'NO1'

fig, ax = plt.subplots()
observed_nps[bidding_zone].plot(ax = ax, color='tomato')
basecase_nps[bidding_zone].plot(ax=ax, color='lightblue')
plt.legend(['Basecase', 'Observed'])
plt.ylabel('Net Postitoin [MWh/h]')
plt.title(bidding_zone)

# Linearized and Observed Border Flows

In [ ]:
# Select a border
from_area = 'NO2'
to_area = 'NO1'

### Observed Flow
The observed flow is retrieved fron ENTSO-E using the fetch_observed_entsoe_data_for_cnec from the fbmc_quality package.

In [ ]:
from fbmc_quality.entsoe_data import fetch_observed_entsoe_data_for_cnec

observed_flow = fetch_observed_entsoe_data_for_cnec(from_area=from_area, to_area=to_area, start_date=start, end_date=end).squeeze()

observed_flow.plot(title='Observed flow', color='r')

### Calculate the linearized flow
The linearized flow is calculated by combining the flow based parameters (PTDFs and Fref) and the observed net positions.
$ F = F_{ref0} + \sum_i(PTDF_i NP_i)$
Where $i$ is the bidding zones.

In [ ]:
cnec_name = f'{from_area}->{to_area}' # Select a CNEC

cnec_data = df_jao_cnec = df_jao[df_jao['cnecName']==cnec_name].droplevel(0) # Extract the cnec data from jao frame

# flow = f_ref + sum(PTDF_i * NP_i)
linearized_flow = '...'

linearized_flow.plot(title='Linearised Flow')

#### Calculate the error between the observed and linearized flow

In [ ]:
linearized_error = '...'

#### Plot the linearisation error

In [1]:
# Create visualiztions of the linearisation error